In [ ]:
# Code 2.1.3
!pip install pandas
from google.colab import files

uploaded = files.upload()
import pandas as pd
train_df = pd.read_csv("train.csv")
train_df.head()

# Kiểm tra thông tin tổng quát
train_df.info()

# Kiểm tra dữ liệu trùng
train_df.duplicated().sum()

# Loại bỏ dữ liệu trùng (nếu có)
train_df = train_df.drop_duplicates()

# Kiểm tra dữ liệu thiếu
train_df.isnull().sum()

# Thay thế giá trị thiếu bằng trung vị hoặc mode
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].median())
train_df['Embarked'] = train_df['Embarked'].fillna(train_df['Embarked'].mode()[0])

# Chuyển biến phân loại thành biến giả (One-hot encoding)
train_df = pd.get_dummies(train_df, columns=['Sex', 'Embarked'], drop_first=True)

# Lọc các cột cần thiết (đặc trưng và nhãn)
features = ['Pclass', 'Sex_male', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_Q', 'Embarked_S']
target = 'Survived'

# Tạo tập đặc trưng X và nhãn y
X_train = train_df[features]
y_train = train_df[target]

# Huấn luyện mô hình Decision Tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score
import graphviz

dt = DecisionTreeClassifier(max_depth=3, random_state=42)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_train)

# Kiểm tra độ chính xác của mô hình
print("Accuracy:", accuracy_score(y_train, y_pred))

# Vẽ cây quyết định
dot_data = export_graphviz(dt, out_file=None,
                           feature_names=X_train.columns,
                           class_names=['Not Survived','Survived'],
                           filled=True, rounded=True,
                           proportion=True)

graph = graphviz.Source(dot_data)
graph


In [ ]:
#2.1.4
#==========================================
#Import thư viện
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import export_graphviz
import graphviz


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, confusion_matrix, roc_auc_score, roc_curve, classification_report
)
#==========================================
#Nạp dữ liệu
from google.colab import files
uploaded = files.upload()  # Chọn file zip


import zipfile
with zipfile.ZipFile("diabetes_prediction_dataset.csv.zip", "r") as zip_ref:
    zip_ref.extractall("data_diabetes")


df = pd.read_csv("data_diabetes/diabetes_prediction_dataset.csv")
# Kiểm tra thông tin dữ liệu
print(df.info())
print(df.head())


# Kiểm tra missing values
print("\nMissing values:\n", df.isnull().sum())


# One-hot encoding cho biến phân loại
df = pd.get_dummies(df, columns=['gender', 'smoking_history'], drop_first=True)
#==========================================
#Chia tập Train/Test
X = df.drop("diabetes", axis=1)
y = df["diabetes"]




X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


print("Kích thước tập huấn luyện:", X_train.shape)
print("Kích thước tập kiểm tra:", X_test.shape)
#==========================================
#Tìm tham số tối ưu cho Cây quyết định
params_dt = {"max_depth": [2, 3, 4, 5, 6, 8, 10]}


grid_dt = GridSearchCV(
    estimator=tree.DecisionTreeClassifier(random_state=42),
    param_grid=params_dt,
    scoring="roc_auc",
    cv=4,
    return_train_score=True,
    verbose=1
)
grid_dt.fit(X_train, y_train)


print("\nBest max_depth:", grid_dt.best_params_)


#Lưu kết quả tìm tham số
cv_dt_df = pd.DataFrame(grid_dt.cv_results_)


plt.figure(figsize=(7, 4))
plt.plot(cv_dt_df["param_max_depth"], cv_dt_df["mean_train_score"], '-o', label='Train ROC AUC', color='orange')
plt.plot(cv_dt_df["param_max_depth"], cv_dt_df["mean_test_score"], '-o', label='Test ROC AUC', color='blue')
plt.fill_between(
    cv_dt_df["param_max_depth"],
    cv_dt_df["mean_test_score"] - cv_dt_df["std_test_score"] / np.sqrt(4),
    cv_dt_df["mean_test_score"] + cv_dt_df["std_test_score"] / np.sqrt(4),
    color='blue', alpha=0.1
)
plt.title("So sánh Train/Test ROC AUC theo độ sâu cây quyết định", fontsize=12)
plt.xlabel("max_depth")
plt.ylabel("ROC AUC")
plt.legend()
plt.tight_layout()
plt.savefig("decision_tree_depth_comparison.png", dpi=300)
plt.show()




#==========================================
#Mô hình Cây quyết định
dt = tree.DecisionTreeClassifier(max_depth=4, random_state=42)
dt.fit(X_train, y_train)


y_pred_dt = dt.predict(X_test)
acc_dt = accuracy_score(y_test, y_pred_dt)
roc_dt = roc_auc_score(y_test, dt.predict_proba(X_test)[:, 1])


print("\n--- Kết quả Cây quyết định cơ bản ---")
print("Độ chính xác (Accuracy):", round(acc_dt, 4))
print("ROC AUC:", round(roc_dt, 4))
# Vẽ cây quyết định
dot_data = export_graphviz(
    dt,
    out_file=None,
    feature_names=X.columns,
    class_names=["Không bệnh", "Bệnh"],
    filled=True,
    rounded=True,
    proportion=True,
    impurity=True,
    precision=2
)
graph = graphviz.Source(dot_data)
graph.render("decision_tree", format="png")  # Xuất ảnh cây quyết định
display(graph)


#==========================================
#Mô hình Rừng cây (Random Forest)


rf = RandomForestClassifier(
    n_estimators=50, max_depth=5, random_state=42, criterion="gini"
)
rf.fit(X_train, y_train)


y_pred_rf = rf.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)
roc_rf = roc_auc_score(y_test, rf.predict_proba(X_test)[:, 1])


print("\n--- Kết quả Rừng cây cơ bản ---")
print("Accuracy (RF):", round(acc_rf, 4))
print("ROC AUC (RF):", round(roc_rf, 4))
#==========================================
#Tìm tham số tối ưu cho Rừng cây
params_rf = {"n_estimators": list(range(10, 110, 10))}


grid_rf = GridSearchCV(
    estimator=RandomForestClassifier(max_depth=5, random_state=42),
    param_grid=params_rf,
    scoring="roc_auc",
    cv=4,
    return_train_score=True,
    verbose=1
)
grid_rf.fit(X_train, y_train)


print("\nBest params (Random Forest):", grid_rf.best_params_)


cv_rf_df = pd.DataFrame(grid_rf.cv_results_)


fig, axs = plt.subplots(1, 2, figsize=(8, 3))
axs[0].plot(cv_rf_df["param_n_estimators"], cv_rf_df["mean_fit_time"], "-o")
axs[0].set_xlabel("Số cây (n_estimators)")
axs[0].set_ylabel("Thời gian huấn luyện trung bình (s)")


axs[1].errorbar(
    cv_rf_df["param_n_estimators"],
    cv_rf_df["mean_test_score"],
    yerr=cv_rf_df["std_test_score"] / np.sqrt(4),
    fmt="-o"
)
axs[1].set_xlabel("Số cây (n_estimators)")
axs[1].set_ylabel("Mean Testing ROC AUC ± 1 SE")
plt.tight_layout()
plt.savefig("random_forest_n_estimators_tuning.png", dpi=300)
plt.show()


# ==========================================
# Phân tích tầm quan trọng đặc trưng
feat_imp = pd.DataFrame({
    "Feature": X.columns,
    "Importance": grid_rf.best_estimator_.feature_importances_
}).sort_values("Importance", ascending=False)


plt.figure(figsize=(7, 4))
sns.barplot(x="Importance", y="Feature", data=feat_imp, palette="viridis")
plt.title("Mức độ quan trọng của các đặc trưng trong mô hình Rừng cây")
plt.tight_layout()
plt.savefig("feature_importance_rf.png", dpi=300)
plt.show()


#==========================================
#Biểu đồ ROC & Ma trận nhầm lẫn


# ROC Curve
fpr, tpr, _ = roc_curve(y_test, rf.predict_proba(X_test)[:, 1])
plt.plot(fpr, tpr, label=f"Random Forest (AUC = {roc_rf:.2f})")
plt.plot([0, 1], [0, 1], "--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Random Forest")
plt.legend()
plt.tight_layout()
plt.savefig("roc_curve_rf.png", dpi=300)
plt.show()


# Ma trận nhầm lẫn
cm = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Dự đoán")
plt.ylabel("Thực tế")
plt.title("Ma trận nhầm lẫn - Random Forest")
plt.tight_layout()
plt.savefig("confusion_matrix_rf.png", dpi=300)
plt.show()


In [1]:
#2.2.3
# 1) Import
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

# 2) Load data
import pandas as pd
df = pd.read_csv("diabetes_prediction_dataset.csv")
X = df[["bmi", "HbA1c_level"]].astype(float).to_numpy()
y = df["diabetes"].astype(int).to_numpy()

# 3) Train/test split (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=101)

# 4) Model (Linear SVM)
from sklearn import svm
clf = svm.LinearSVC(C=1.0, dual=False, max_iter=5000).fit(X_train, y_train)

# 5) Predict & evaluate
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
cls_rep = classification_report(
    y_test, y_pred,

    labels=[0, 1],
    target_names=["No diabetes", "Diabetes"],
)

# 6) Print
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


FileNotFoundError: [Errno 2] No such file or directory: 'diabetes_prediction_dataset.csv'

In [ ]:
#2.2.4
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import joblib


# --- 1. Đọc dữ liệu ---
df = pd.read_csv("data.csv")


# Bỏ các hàng bị thiếu giá trị
df = df.dropna()


# --- 2. Mã hóa dữ liệu chữ thành số ---
encoder = LabelEncoder()


for col in ['AnimalName', 'symptoms1', 'symptoms2', 'symptoms3', 'symptoms4', 'symptoms5', 'Dangerous']:
    df[col] = encoder.fit_transform(df[col])


# --- 3. Tách biến đầu vào và đầu ra ---
X = df.drop('Dangerous', axis=1)
y = df['Dangerous']


# --- 4. Chia dữ liệu train/test ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


# --- 5. Chuẩn hóa dữ liệu ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# --- 6. Huấn luyện mô hình SVM ---
model = SVC(kernel='rbf', C=1, gamma='scale', random_state=42)
model.fit(X_train_scaled, y_train)


# --- 7. Đánh giá ---
y_pred = model.predict(X_test_scaled)
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification report:\n", classification_report(y_test, y_pred))


# --- 8. Lưu model và scaler ---
joblib.dump(model, "svm_animal_model.pkl")
joblib.dump(scaler, "scaler.pkl")


print("✅ Huấn luyện xong! Mô hình đã lưu thành công.")



In [ ]:
#2.3.3
# Import thư viện
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
# Đọc dữ liệu
data = pd.read_csv('Customer_Behaviour.csv', encoding='latin-1')
# Hiển thị 5 dòng đầu
data.head()
print(data.columns)
# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Mã hóa cột Gender: Male/Female -> 1/0
le = LabelEncoder()
data['Gender'] = le.fit_transform(data['Gender'])
print(data.head())

In [ ]:
# Chia X (đặc trưng) và y (nhãn)
X = data.drop('Purchased', axis=1)   # Gender, Age, Salary
y = data['Purchased']                # Nhãn: Purchased
# Tách train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
# Khởi tạo mô hình Naïve Bayes (Gaussian)
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

# Huấn luyện
model.fit(X_train, y_train)

# Dự đoán
y_pred = model.predict(X_test)

In [ ]:
# Độ chính xác
print("Accuracy:", accuracy_score(y_test, y_pred))

# Ma trận nhầm lẫn
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Báo cáo chi tiết
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# Code 2.3.4
# Mã hóa dữ liệu
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
for col in data.columns:
    data[col] = labelencoder.fit_transform(data[col])

# Xem 5 dòng đầu tiên
print(data.head())

# Tách biến độc lập và phụ thuộc
X = data.drop('class', axis=1)   # class: edible=e, poisonous=p
y = data['class']

# Chia dữ liệu train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=42)

# Huấn luyện mô hình Naive Bayes
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train, y_train)

# Dự đoán và đánh giá mô hình
y_pred = model.predict(X_test)
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Hiển thị kết quả
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
